<a href="https://colab.research.google.com/github/alaabakk/E2504_bachelor/blob/main/Models/Training/ModelTrainingOptuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # Mounts Google Drive to access files stored in your Drive

In [ ]:
!pip install ultralytics optuna roboflow # Installs required packages for YOLO training, hyperparameter optimization, and dataset handling

In [ ]:
!nvidia-smi # Displays GPU information

from ultralytics import YOLO # Imports the YOLO class from the Ultralytics package
import os # Imports the os module for interacting with the operating system
!yolo checks # Runs a check to verify the YOLO environment and dependencies

In [ ]:
#Dataset downloaded from RoboFlow, this snippet can be found in your RoboFlow user with the correct API_key and project workspace

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("YOUR_API_WORKSPACE").project("YOUR_API_PROJECT")
version = project.version(1) # Replace with the version number you want to download
dataset = version.download("yolov8") 

In [ ]:
# Imports
import os
import time
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from optuna.pruners  import SuccessiveHalvingPruner
from datetime import datetime as dt
from zoneinfo import ZoneInfo as zi

# Create root directory for this training session
# Output folder is timestamped for uniqueness and traceability
drive_base = "/content/drive/MyDrive/Your_Folder" # Change this to the folder where you want to save your results
os.makedirs(drive_base, exist_ok=True)

timestamp = dt.now(zi("Europe/Oslo")).strftime("%Y%m%d_%H%M")
model_file = "yolov8n.pt" # Chose the model you want to train, yolov8n.pt is the smallest and fastest model. You can also use yolov8s.pt, yolov8m.pt, yolov8l.pt, or yolov8x.pt for larger models
base_name  = os.path.splitext(os.path.basename(model_file))[0]
root_dir   = os.path.join(drive_base, f"{base_name}_{timestamp}")
os.makedirs(root_dir, exist_ok=True)

print(f"All output will be saved to: {root_dir}")

# Define the Optuna objective function
def objective(trial):
    # 1) Suggest hyperparameters within defined search space
    lr0          = trial.suggest_float("lr0",          1e-5, 1e-1, log=True) 
    momentum     = trial.suggest_float("momentum",     0.6,  0.98)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    optimizer    = trial.suggest_categorical("optimizer", ["SGD","Adam"])
    batch        = trial.suggest_int("batch",           8,   32)

    # 2) Load model and define pruning callback based on validation mAP
    model = YOLO(model_file)
    def prune_callback(trainer):
        epoch = trainer.epoch
        score = trainer.metrics.get("metrics/mAP50-95(B)")
        if score is None:
            return
        trial.report(score, step=epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    model.add_callback("on_fit_epoch_end", prune_callback)

    # 3) Train model using suggested hyperparameters
    run_name = f"training{trial.number}"
    results = model.train(
        data         = "/content/Your_data.yaml_file", #Path to data.yaml file uploaded from RoboFlow
        epochs       = 300, # Number of epochs to train
        imgsz        = 640, # Image size for training
        lr0          = lr0, 
        momentum     = momentum,
        weight_decay = weight_decay,
        optimizer    = optimizer,
        batch        = batch,
        project      = root_dir,
        name         = run_name,
        exist_ok     = False,
        verbose      = False
    )

    # 4) Read the results CSV and return the best mAP50-95
    results_csv = os.path.join(results.save_dir, "results.csv")
    for _ in range(5):
        if os.path.exists(results_csv):
            break
        time.sleep(1)
    df = pd.read_csv(results_csv)
    return df["metrics/mAP50-95(B)"].max()

# 5) Run the Optuna hyperparameter study
if __name__ == "__main__":
    sampler = TPESampler()
    pruner  = SuccessiveHalvingPruner()
    db_path = os.path.join(root_dir, f"optuna_study.db")

    study = optuna.create_study(
        study_name = "yolo_study",
        storage    = f"sqlite:///{db_path}",
        sampler    = sampler,
        pruner     = pruner,
        direction  = "maximize",
        load_if_exists = True
    )
    study.optimize(objective, n_trials=100) # Number of trials to run, with TPE sampler and pruning bewteen 100 and 1000 is recommended

    # 6) Save and print results
    best_trial = study.best_trial
    best_training_name = f"training{best_trial.number}"

    print("Best hyperparameters:", best_trial.params)
    print(f"Best mAP50-95: {best_trial.value:.4f}")
    print(f"Best training run: {best_training_name}")

    # Save all trial history to CSV
    study.trials_dataframe().to_csv(os.path.join(root_dir, "optuna_trials.csv"), index=False)
    print(f"Optuna history saved to: {os.path.join(root_dir, 'optuna_trials.csv')}")


